<a href="https://colab.research.google.com/github/henriquedavid/ENEM-Analysis/blob/master/Data_People.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações 

In [2]:
import os
import folium
import json
import pandas as pd
from branca.colormap import linear
import numpy as np
from shapely.geometry import Polygon
from shapely.geometry import Point
from numpy import random
import dask.dataframe as dd

# Leitura dos arquivos

A seguinte célula representa a leitura dos arquivos

In [116]:
data_2013 =  pd.read_csv("data/2013/DADOS/MICRODADOS_ENEM_2013.csv", encoding='latin1', nrows=250000,
                         usecols= geral+var13_14, engine='python', delimiter=';')
data_2014 =  pd.read_csv("data/2014/microdados_enem2014/DADOS/MICRODADOS_ENEM_2014.csv",
                         usecols= geral+var13_14, encoding='latin1', nrows=250000, engine='python')
data_2012 =  pd.read_csv("data/2012/DADOS/DADOS_ENEM_2012.csv", encoding='latin1', nrows=250000, usecols= geral+var12)
data_2012["NU_NOTA_REDACAO"] = data_2012["NU_NOTA_REDACAO"].replace(".", "0")

data_2012_quest =  pd.read_csv("data/2012/DADOS/QUESTIONARIO_ENEM_2012.csv", encoding='latin1', nrows=250000,
                               usecols= ["Q03"])
data_2012 = pd.concat([data_2012, data_2012_quest], axis=1)

print(data_2014.shape)
    
data_2012.head(10)

(250000, 27)


,NU_INSCRICAO,IDADE,TP_SEXO,UF_INSC,ST_CONCLUSAO,ANO_CONCLUIU,TP_ESCOLA,TP_COR_RACA,IN_BAIXA_VISAO,IN_CEGUEIRA,...,IN_LACTANTE,IN_IDOSO,IN_AUTISMO,IN_SABATISTA,NU_NT_CN,NU_NT_CH,NU_NT_LC,NU_NT_MT,NU_NOTA_REDACAO,Q03
0,400000000001,40,0,BA,1,2003,.,3,0,0,...,0,0,0,0,.,.,.,.,0,D
1,400000000002,18,0,PE,1,2011,.,1,0,0,...,0,0,0,0,537.7,552.2,532.6,468.3,280,G
2,400000000003,20,1,RJ,1,2011,.,3,0,0,...,0,0,0,0,.,.,.,.,0,C
3,400000000004,30,0,DF,1,2010,.,3,0,0,...,0,0,0,0,459,557.7,545,565.3,720,F
4,400000000005,20,0,RJ,1,2010,.,1,0,0,...,0,0,0,0,.,.,.,.,0,C
5,400000000006,21,1,BA,1,2010,.,2,0,0,...,0,0,0,0,.,.,.,.,0,C
6,400000000007,20,0,PB,1,2009,.,1,0,0,...,0,0,0,0,507.3,565.1,500.6,659.7,380,D
7,400000000008,26,0,MG,1,2004,.,1,0,0,...,0,0,0,0,522.8,633.6,.,.,0,B
8,400000000009,24,1,GO,1,2006,.,3,0,0,...,0,0,0,0,.,.,.,.,0,D
9,400000000010,18,1,MG,1,2010,.,1,0,0,...,0,0,0,0,590.3,611.3,552.4,620.7,500,D


Uma observação importante é que as respostas do questionário de inscrição é contido em outro dataset, portanto, basta lê-lo e concatená-lo com o lido anteriormente 

In [141]:
data_2012_quest =  pd.read_csv("data/2012/DADOS/QUESTIONARIO_ENEM_2012.csv", encoding='latin1', nrows=250000,
                               usecols= ["Q03"])
data_2012 = pd.concat([data_2012, data_2012_quest], axis=1)

data_2012["NU_NOTA_REDACAO"] = data_2012["NU_NOTA_REDACAO"].replace(".", "0")

In [142]:
print (data_2012.columns[0:40])
data_2012.shape

Index(['NU_INSCRICAO', 'IDADE', 'TP_SEXO', 'UF_INSC', 'ST_CONCLUSAO',
       'ANO_CONCLUIU', 'TP_ESCOLA', 'TP_COR_RACA', 'IN_BAIXA_VISAO',
       'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DEFICIENCIA_AUDITIVA',
       'IN_SURDO_CEGUEIRA', 'IN_DEFICIENCIA_FISICA', 'IN_DEFICIENCIA_MENTAL',
       'IN_DEFICIT_ATENCAO', 'IN_DISLEXIA', 'IN_GESTANTE', 'IN_LACTANTE',
       'IN_IDOSO', 'IN_AUTISMO', 'IN_SABATISTA', 'NU_NT_CN', 'NU_NT_CH',
       'NU_NT_LC', 'NU_NT_MT', 'NU_NOTA_REDACAO', 'Q03', 'Q03'],
      dtype='object')


(250000, 29)

# Análise de participantes com necessidades especiais

Segundo o enem, a seguinte lista representa as pessoas que indicam que possuem necessidades especias no ato de inscrição :
"IN_BAIXA_VISAO", "IN_CEGUEIRA", "IN_SURDEZ", "IN_DEFICIENCIA_AUDITIVA", "IN_SURDO_CEGUEIRA", "IN_DEFICIENCIA_FISICA", "IN_DEFICIENCIA_MENTAL", "IN_DEFICIT_ATENCAO","IN_DISLEXIA","IN_GESTANTE", "IN_LACTANTE", "IN_IDOSO", "IN_AUTISMO", "IN_SABATISTA"

In [143]:
lista_necessidade =  ["IN_BAIXA_VISAO", "IN_CEGUEIRA", "IN_SURDEZ", "IN_DEFICIENCIA_AUDITIVA", "IN_SURDO_CEGUEIRA", 
                      "IN_DEFICIENCIA_FISICA","IN_DEFICIENCIA_MENTAL","IN_DEFICIT_ATENCAO","IN_DISLEXIA","IN_GESTANTE",
                      "IN_LACTANTE","IN_IDOSO","IN_AUTISMO","IN_SABATISTA"]
lista_necessidade

['IN_BAIXA_VISAO',
 'IN_CEGUEIRA',
 'IN_SURDEZ',
 'IN_DEFICIENCIA_AUDITIVA',
 'IN_SURDO_CEGUEIRA',
 'IN_DEFICIENCIA_FISICA',
 'IN_DEFICIENCIA_MENTAL',
 'IN_DEFICIT_ATENCAO',
 'IN_DISLEXIA',
 'IN_GESTANTE',
 'IN_LACTANTE',
 'IN_IDOSO',
 'IN_AUTISMO',
 'IN_SABATISTA']

Selecionando os usuários com necessidades especiais

In [144]:
l2012 =[]
for i in lista_necessidade:
    l2012.append(data_2012[data_2012[i] == 1])
l2013 =[]
for i in lista_necessidade:
    l2013.append(data_2013[data_2013[i] == 1])
l2014 =[]
for i in lista_necessidade:
    l2014.append(data_2014[data_2014[i] == 1])
    
l2012

[        NU_INSCRICAO  IDADE  TP_SEXO UF_INSC  ST_CONCLUSAO ANO_CONCLUIU  \
 2161    400000002162     48        1      RJ             1         2005   
 8035    400000008036     50        1      CE             1         2003   
 11457   400000011458     33        0      RJ             1         2006   
 11789   400000011790     29        1      SP             1         2003   
 11993   400000011994     26        0      CE             1         2004   
 13253   400000013254     45        0      SP             1         2003   
 13552   400000013553     56        0      SP             1         2004   
 14798   400000014799     17        1      RJ             1         2011   
 18346   400000018347     37        1      RJ             1         2003   
 21435   400000021436     19        1      RN             1         2011   
 21910   400000021911     29        0      SP             1         2003   
 22271   400000022272     38        0      SP             4            .   
 23632   400

Removendo usuários com mais de uma necessidade

In [39]:
df_lista_necessidade_2012 = pd.concat(l2012)
df_lista_necessidade_2012["NU_NOTA_REDACAO"] = df_lista_necessidade_2012["NU_NOTA_REDACAO"].replace(".", "0")
df_lista_necessidade_2012 = df_lista_necessidade_2012[~df_lista_necessidade_2012.duplicated()]
print(df_lista_necessidade_2012.shape)
df_lista_necessidade_2013 = pd.concat(l2013)
df_lista_necessidade_2013 = df_lista_necessidade_2013[~df_lista_necessidade_2013.duplicated()]
print(df_lista_necessidade_2013.shape)
df_lista_necessidade_2014 = pd.concat(l2014)
df_lista_necessidade_2014 = df_lista_necessidade_2014[~df_lista_necessidade_2014.duplicated()]
print(df_lista_necessidade_2014.shape)

(3395, 27)
(4514, 27)
(4243, 27)


Selecionando alunos que não possuem necessidades especiais

In [40]:
df_lista_sem_necessidade_2012 = data_2012[~data_2012.NU_INSCRICAO.isin(df_lista_necessidade_2012.NU_INSCRICAO)]
df_lista_sem_necessidade_2013 = data_2013[~data_2013.NU_INSCRICAO.isin(df_lista_necessidade_2013.NU_INSCRICAO)]
df_lista_sem_necessidade_2014 = data_2014[~data_2014.NU_INSCRICAO.isin(df_lista_necessidade_2014.NU_INSCRICAO)]
print(df_lista_sem_necessidade_2012.shape)
print(df_lista_sem_necessidade_2013.shape)
print(df_lista_sem_necessidade_2014.shape)

(246605, 27)
(245486, 27)
(245757, 27)


In [138]:
print(np.asarray(df_lista_necessidade_2012["NU_NOTA_REDACAO"], dtype=np.float).mean())
print(np.asarray(df_lista_sem_necessidade_2012["NU_NOTA_REDACAO"], dtype=np.float).mean())
print(np.asarray(df_lista_necessidade_2013["NU_NOTA_REDACAO"], dtype=np.float).mean())
print(np.asarray(df_lista_sem_necessidade_2013["NU_NOTA_REDACAO"], dtype=np.float).mean())
print(np.asarray(df_lista_necessidade_2014["NU_NOTA_REDACAO"], dtype=np.float).mean())
print(np.asarray(df_lista_sem_necessidade_2014["NU_NOTA_REDACAO"], dtype=np.float).mean())
#df_lista_sem_necessidade_2013.NU.value_counts()

374.33284241531663
389.65829565499484
280.3196721311475
330.6886747105741
361.8522271977375
396.3436239862954


## Gráfico bloxplot comparativo das notas das redações e provas objetivas dos ENEM 2012-2014

In [109]:
import plotly.plotly as py
import plotly.graph_objs as go

data = []
listas = [[df_lista_necessidade_2012,"2012", df_lista_sem_necessidade_2012], 
          [df_lista_necessidade_2013,"2013", df_lista_sem_necessidade_2013],
          [df_lista_necessidade_2014,"2014", df_lista_sem_necessidade_2014]]
for lista in listas:
    data.append(go.Box(
        y=lista[0].NU_NOTA_REDACAO.tolist(),
        name = 'Com necessidade '+lista[1],
        boxmean=True

    ))
    data.append(go.Box(
        y=lista[2].NU_NOTA_REDACAO.tolist(),
        name = 'Sem necessidade '+lista[1] ,
        boxmean=True
    ),)
    
layout = go.Layout(
    title = "Notas redação ENEM 2012-2014"
)
url = py.plot({"data": data, "layout": layout}, filename='Redação boxplot', title="Notas Redação")

/home/daniel/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



In [110]:
data = []
listas = [[df_lista_necessidade_2012,"2012", df_lista_sem_necessidade_2012], 
          [df_lista_necessidade_2013,"2013", df_lista_sem_necessidade_2013],
          [df_lista_necessidade_2014,"2014", df_lista_sem_necessidade_2014]]
for lista in listas:
    if(lista[1] == "2012"):
        data.append(go.Box(
            y=lista[0].NU_NT_CN.tolist(),
            name = 'Com necessidade '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[2].NU_NT_CN.tolist(),
            name = 'Sem necessidade '+lista[1] ,
            boxmean=True
        ),)
    else:
        data.append(go.Box(
            y=lista[0].NOTA_CN.tolist(),
            name = 'Com necessidade '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[2].NOTA_CN.tolist(),
            name = 'Sem necessidade '+lista[1] ,
            boxmean=True
        ),)
    
layout = go.Layout(
    title = "Notas ciências da natureza ENEM 2012-2014"
)
url = py.plot({"data": data, "layout": layout}, filename='CN boxplot')

/home/daniel/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



In [111]:
data = []
listas = [[df_lista_necessidade_2012,"2012", df_lista_sem_necessidade_2012], 
          [df_lista_necessidade_2013,"2013", df_lista_sem_necessidade_2013],
          [df_lista_necessidade_2014,"2014", df_lista_sem_necessidade_2014]]
for lista in listas:
    if(lista[1] == "2012"):
        data.append(go.Box(
            y=lista[0].NU_NT_CH.tolist(),
            name = 'Com necessidades '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[2].NU_NT_CH.tolist(),
            name = 'Sem necessidades '+lista[1] ,
            boxmean=True
        ),)
    else:
        data.append(go.Box(
            y=lista[0].NOTA_CH.tolist(),
            name = 'Com necessidades '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[2].NOTA_CH.tolist(),
            name = 'Sem necessidades '+lista[1] ,
            boxmean=True
        ),)
    
layout = go.Layout(
    title = "Notas ciências humanas ENEM 2012-2014"
)
url = py.plot({"data": data, "layout": layout}, filename='CH boxplot')

/home/daniel/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



In [112]:
data = []
listas = [[df_lista_necessidade_2012,"2012", df_lista_sem_necessidade_2012], 
          [df_lista_necessidade_2013,"2013", df_lista_sem_necessidade_2013],
          [df_lista_necessidade_2014,"2014", df_lista_sem_necessidade_2014]]
for lista in listas:
    if(lista[1] == "2012"):
        data.append(go.Box(
            y=lista[0].NU_NT_LC.tolist(),
            name = 'Com necessidades '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[2].NU_NT_LC.tolist(),
            name = 'Sem necessidades '+lista[1] ,
            boxmean=True
        ),)
    else:
        data.append(go.Box(
            y=lista[0].NOTA_LC.tolist(),
            name = 'Com necessidades '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[2].NOTA_LC.tolist(),
            name = 'Sem necessidades '+lista[1] ,
            boxmean=True
        ),)
    
layout = go.Layout(
    title = "Notas linguagens e códigos ENEM 2012-2014"
)
url = py.plot({"data": data, "layout": layout}, filename='LC boxplot')

/home/daniel/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



In [113]:
data = []
listas = [[df_lista_necessidade_2012,"2012", df_lista_sem_necessidade_2012], 
          [df_lista_necessidade_2013,"2013", df_lista_sem_necessidade_2013],
          [df_lista_necessidade_2014,"2014", df_lista_sem_necessidade_2014]]
for lista in listas:
    if(lista[1] == "2012"):
        data.append(go.Box(
            y=lista[0].NU_NT_MT.tolist(),
            name = 'Com necessidades '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[2].NU_NT_MT.tolist(),
            name = 'Sem necessidades '+lista[1] ,
            boxmean=True
        ),)
    else:
        data.append(go.Box(
            y=lista[0].NOTA_MT.tolist(),
            name = 'Com necessidades '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[2].NOTA_MT.tolist(),
            name = 'Sem necessidades '+lista[1] ,
            boxmean=True
        ),)
    
layout = go.Layout(
    title = "Notas Matemática ENEM 2012-2014"
)
url = py.plot({"data": data, "layout": layout}, filename='MT boxplot')

/home/daniel/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



## Gráfico bloxplot comparativo das notas das redações e provas objetivas dos ENEM 2012-2014 levando em conta a renda 


Selecionando usuários com baixa renda

In [145]:
baixa_renda = ["A","B","C"]
data_2014[~data_2014.Q003.isin(baixa_renda)].head()

,NU_INSCRICAO,UF_RESIDENCIA,IDADE,TP_SEXO,ST_CONCLUSAO,ANO_CONCLUIU,TP_ESCOLA,TP_COR_RACA,IN_BAIXA_VISAO,IN_CEGUEIRA,...,IN_LACTANTE,IN_IDOSO,IN_AUTISMO,IN_SABATISTA,NOTA_CN,NOTA_CH,NOTA_LC,NOTA_MT,NU_NOTA_REDACAO,Q003
0,140000000001,SP,18.0,F,1,2013.0,1.0,1,0,0,...,0,0,0,0,536.9,570.2,576.1,478.3,560,E
2,140000000003,SP,19.0,M,1,2012.0,1.0,2,0,0,...,0,0,0,0,575.9,634.7,541.5,617.6,600,K
4,140000000005,BA,45.0,M,1,2004.0,2.0,1,0,0,...,0,0,0,0,612.6,700.7,596.5,648.2,580,F
8,140000000009,MG,19.0,M,1,2012.0,1.0,1,0,0,...,0,0,0,0,518.8,569.8,561.0,623.2,640,F
10,140000000011,DF,22.0,F,1,2010.0,2.0,1,0,0,...,0,0,0,0,657.0,693.5,647.4,660.6,720,H


In [131]:
data = []
listas = [[data_2012,"2012"], 
          [data_2013,"2013"],
          [data_2014,"2014"]]
baixa_renda = ["A","B","C"]
for lista in listas:
    if(lista[1] == "2012"):
        data.append(go.Box(
            y=lista[0][lista[0]["Q03"].isin(baixa_renda)].NU_NOTA_REDACAO.tolist(),
            name = 'Renda <= 1.5  '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[0][~lista[0]["Q03"].isin(baixa_renda)].NU_NOTA_REDACAO.tolist(),
            name = 'Renda > 1.5 '+lista[1] ,
            boxmean=True
        ),)
    else:
        data.append(go.Box(
            y=lista[0][lista[0]["Q003"].isin(baixa_renda)].NU_NOTA_REDACAO.tolist(),
            name = 'Renda <= 1.5 '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[0][~lista[0]["Q003"].isin(baixa_renda)].NU_NOTA_REDACAO.tolist(),
            name = 'Renda > 1.5 '+lista[1] ,
            boxmean=True
        ),)
    
layout = go.Layout(
    title = "Notas da Redação por renda ENEM 2012-2014"
)
url = py.plot({"data": data, "layout": layout}, filename='Renda boxplot')

/home/daniel/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



In [134]:
data = []
listas = [[data_2012,"2012"], 
          [data_2013,"2013"],
          [data_2014,"2014"]]
baixa_renda = ["A","B","C"]
for lista in listas:
    if(lista[1] == "2012"):
        data.append(go.Box(
            y=lista[0][lista[0]["Q03"].isin(baixa_renda)].NU_NT_CN.tolist(),
            name = 'Renda <= 1.5  '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[0][~lista[0]["Q03"].isin(baixa_renda)].NU_NT_CN.tolist(),
            name = 'Renda > 1.5 '+lista[1] ,
            boxmean=True
        ),)
    else:
        data.append(go.Box(
            y=lista[0][lista[0]["Q003"].isin(baixa_renda)].NOTA_CN.tolist(),
            name = 'Renda <= 1.5 '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[0][~lista[0]["Q003"].isin(baixa_renda)].NOTA_CN.tolist(),
            name = 'Renda > 1.5 '+lista[1] ,
            boxmean=True
        ),)
    
layout = go.Layout(
    title = "Notas ciências da natureza por renda ENEM 2012-2014"
)
url = py.plot({"data": data, "layout": layout}, filename='Renda CN boxplot')

/home/daniel/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



In [135]:
data = []
listas = [[data_2012,"2012"], 
          [data_2013,"2013"],
          [data_2014,"2014"]]
baixa_renda = ["A","B","C"]
for lista in listas:
    if(lista[1] == "2012"):
        data.append(go.Box(
            y=lista[0][lista[0]["Q03"].isin(baixa_renda)].NU_NT_CH.tolist(),
            name = 'Renda <= 1.5  '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[0][~lista[0]["Q03"].isin(baixa_renda)].NU_NT_CH.tolist(),
            name = 'Renda > 1.5 '+lista[1] ,
            boxmean=True
        ),)
    else:
        data.append(go.Box(
            y=lista[0][lista[0]["Q003"].isin(baixa_renda)].NOTA_CH.tolist(),
            name = 'Renda <= 1.5 '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[0][~lista[0]["Q003"].isin(baixa_renda)].NOTA_CH.tolist(),
            name = 'Renda > 1.5 '+lista[1] ,
            boxmean=True
        ),)
    
layout = go.Layout(
    title = "Notas ciências humanas por renda ENEM 2012-2014"
)
url = py.plot({"data": data, "layout": layout}, filename='Renda CH boxplot')

/home/daniel/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



In [133]:
data = []
listas = [[data_2012,"2012"], 
          [data_2013,"2013"],
          [data_2014,"2014"]]
baixa_renda = ["A","B","C"]
for lista in listas:
    if(lista[1] == "2012"):
        data.append(go.Box(
            y=lista[0][lista[0]["Q03"].isin(baixa_renda)].NU_NT_LC.tolist(),
            name = 'Renda <= 1.5  '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[0][~lista[0]["Q03"].isin(baixa_renda)].NU_NT_LC.tolist(),
            name = 'Renda > 1.5 '+lista[1] ,
            boxmean=True
        ),)
    else:
        data.append(go.Box(
            y=lista[0][lista[0]["Q003"].isin(baixa_renda)].NOTA_LC.tolist(),
            name = 'Renda <= 1.5 '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[0][~lista[0]["Q003"].isin(baixa_renda)].NOTA_LC.tolist(),
            name = 'Renda > 1.5 '+lista[1] ,
            boxmean=True
        ),)
    
layout = go.Layout(
    title = "Notas linguagens e códigos por renda ENEM 2012-2014"
)
url = py.plot({"data": data, "layout": layout}, filename='Renda LC boxplot')

/home/daniel/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



In [136]:
data = []
listas = [[data_2012,"2012"], 
          [data_2013,"2013"],
          [data_2014,"2014"]]
baixa_renda = ["A","B","C"]
for lista in listas:
    if(lista[1] == "2012"):
        data.append(go.Box(
            y=lista[0][lista[0]["Q03"].isin(baixa_renda)].NU_NT_MT.tolist(),
            name = 'Renda <= 1.5  '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[0][~lista[0]["Q03"].isin(baixa_renda)].NU_NT_MT.tolist(),
            name = 'Renda > 1.5 '+lista[1] ,
            boxmean=True
        ),)
    else:
        data.append(go.Box(
            y=lista[0][lista[0]["Q003"].isin(baixa_renda)].NOTA_MT.tolist(),
            name = 'Renda <= 1.5 '+lista[1],
            boxmean=True

        ))
        data.append(go.Box(
            y=lista[0][~lista[0]["Q003"].isin(baixa_renda)].NOTA_MT.tolist(),
            name = 'Renda > 1.5 '+lista[1] ,
            boxmean=True
        ),)
    
layout = go.Layout(
    title = "Notas matemática por renda ENEM 2012-2014"
)
url = py.plot({"data": data, "layout": layout}, filename='Renda MT boxplot')

/home/daniel/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.

